In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv(r"C:\Users\Prajyot Lad\Python\XYZCorp_LendingData.txt",delimiter="\t")
df_rev=pd.DataFrame.copy(df)
df_rev.shape

In [ ]:
df_rev=df_rev.drop(["mths_since_last_record","mths_since_last_major_derog","annual_inc_joint",
            "dti_joint","verification_status_joint","open_acc_6m","open_il_6m",
            "open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il",
            "il_util","open_rv_12m","open_rv_24m","max_bal_bc","all_util",
            "total_rev_hi_lim","inq_fi","total_cu_tl","inq_last_12m",
            "mths_since_last_delinq","id","member_id","emp_title","purpose",
            "title","zip_code","addr_state","desc","emp_length","last_pymnt_d",
            "last_credit_pull_d","next_pymnt_d","pymnt_plan",
            "grade","sub_grade","earliest_cr_line","total_acc"],axis=1)


In [ ]:
df_rev.shape

In [ ]:
#Missing values
df_rev.isnull().sum()

In [ ]:
#Treatment of Missing Values
for value in["revol_util","collections_12_mths_ex_med","tot_coll_amt","tot_cur_bal"]:
    df_rev[value].fillna(df_rev[value].mean(),inplace=True)

In [ ]:
#issue_d is object datatype to make use for split issue_d in date
df_rev.issue_d=pd.to_datetime(df_rev.issue_d) 
col_name='issue_d'
print(df_rev[col_name].dtype)
df_rev.issue_d
df_rev.dtypes


In [ ]:
#converting categorical variable to numeric
colname=[]
for x in df_rev.columns:
    if df_rev[x].dtype=="object":
        colname.append(x)
colname


In [ ]:
# For preprocessing the data
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for x in colname:
    df_rev[x]=le.fit_transform(df_rev[x])

In [ ]:
#split data in test and train*
split_date="Jun-2015"

In [ ]:
df_training=df_rev.loc[df_rev['issue_d']< split_date]
df_training=df_training.drop(['issue_d'],axis=1)
df_training.shape

In [ ]:
df_test=df_rev.loc[df_rev['issue_d']>= split_date]
df_test=df_test.drop(['issue_d'],axis=1)
df_test.shape

In [ ]:
#selecting X and Y
X_train=df_training.values[:,:-1]
Y_train=df_training.values[:,-1]
Y_train=Y_train.astype(int)
print(Y_train)


In [ ]:
X_test=df_test.values[:,:-1]
Y_test=df_test.values[:,-1]
Y_test=Y_test.astype(int)
print(Y_test)

In [ ]:
df_rev=df_rev.drop(["issue_d"],axis=1)

In [ ]:
#Scaling the data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
print(X_test)
print(X_train)


In [ ]:
#1]LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
#create a model building
classifier=LogisticRegression()
#fitting training data to model 
classifier.fit(X_train,Y_train)

In [ ]:
Y_pred=classifier.predict(X_test)
print(list(zip(Y_test,Y_pred)))

In [ ]:
print(classifier.coef_)
print(classifier.intercept_)


In [ ]:
##Confusion matrix
from sklearn.metrics import confusion_matrix,accuracy_score,\
classification_report

In [ ]:
cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

In [ ]:
print("Classification report:")
print(classification_report(Y_test,Y_pred))
acc=accuracy_score(Y_test,Y_pred)
print("Accuracy of the model:",acc)


In [ ]:
#store the predicted values
y_pred_prob=classifier.predict_proba(X_test)
print(y_pred_prob)


In [ ]:
# For adjusting threshold 
for a in np.arange(0.4,0.61,0.01):
    predict_mine = np.where(y_pred_prob[:,1] > a, 1, 0)
    cfm=confusion_matrix(Y_test, predict_mine)
    total_err=cfm[0,1]+cfm[1,0]
    print("Errors at threshold ", a, ":",total_err, " , type 2 error :",
    cfm[1,0]," , type 1 error:", cfm[0,1])

In [ ]:
y_pred_class=[]
for value in y_pred_prob[:,1]:
    if value > 0.45:
        y_pred_class.append(1)
    else:
        
        y_pred_class.append(0)
print(y_pred_class)

In [ ]:
#repeat confusion matrix
from sklearn.metrics import confusion_matrix,accuracy_score,\
classification_report
cfm=confusion_matrix(Y_test,y_pred_class)
print(cfm)
print("Classification report:")
print(classification_report(Y_test,y_pred_class))


In [ ]:
acc=accuracy_score(Y_test,y_pred_class)
print("Accuracy of the model:",acc)

In [ ]:
#ROC
from sklearn import metrics
fpr, tpr, z = metrics.roc_curve(Y_test, y_pred_prob[:,1])
auc = metrics.auc(fpr,tpr)
print(auc)

In [ ]:

import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = auc)
plt.legend(loc = 'lower right')        
plt.plot([0, 1], [0, 1],'r--')  #redline
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
#Note-Adjusting threshold dosent help.
#before Adj thresh recall value for 1= 0.51,after adj thresh = 0.52(0.40)

In [ ]:
#2]Decision Tree
#Model Building(Decision Tree)
from sklearn.tree import DecisionTreeClassifier
model_DT=DecisionTreeClassifier(criterion="gini",random_state=10)
model_DT.fit(X_train,Y_train)
Y_pred=model_DT.predict(X_test)
print(Y_pred)

In [ ]:
#feature imp
print(list(zip(df_rev.columns,model_DT.feature_importances_)))

In [ ]:
#generate file and upload the code in webgraphviz.com to plot the decision tree
#from sklearn import tree
#with open('/Users/Prajyot/Desktop/model_DT.txt','w') as f:
#    f=tree.export_graphviz(model_DT,feature_names=df_rev[:-1],out_file=f)

In [ ]:
#3]SVM
from sklearn import svm
model_DT=svm.SVC(kernel="rbf",gamma=0.1,C=50)
model_DT.fit(X_train,Y_train)
Y_pred=model_DT.predict(X_test)
print(Y_pred)


In [ ]:
#Ensemble Modelling
#4]Bagging
from sklearn.ensemble import ExtraTreesClassifier
model=ExtraTreesClassifier(n_estimators=151,random_state=10)
model.fit(X_train,Y_train)
Y_pred=model.predict(X_test)

In [ ]:
#5]Random Forest
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=101,random_state=10)
model.fit(X_train,Y_train)
Y_pred=model.predict(X_test)

In [ ]:
#6]AdaBoost
from sklearn.ensemble import AdaBoostClassifier
model_AdaBoost=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=50,random_state=10)
model_AdaBoost.fit(X_train,Y_train)
Y_pred=model_AdaBoost.predict(X_test)

In [ ]:
#7]GradientBoost
from sklearn.ensemble import GradientBoostingClassifier
model_GradientBoosting=GradientBoostingClassifier(n_estimators=71,random_state=10)
model_GradientBoosting.fit(X_train,Y_train)
Y_pred=model_GradientBoosting.predict(X_test)

In [ ]:
#8]Ensemble Model(Voting Classifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('log', model1))
model2 = DecisionTreeClassifier(criterion='gini',random_state=10)
estimators.append(('cart', model2))
# = SVC(kernel="rbf", C=70,gamma=0.1)
#estimators.append(('svm', model3))
#model4 = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
#estimators.append(('knn', model4))


In [ ]:
# create the ensemble model
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train,Y_train)
Y_pred=ensemble.predict(X_test)